In [1]:
# from deepiforest.algorithms.dif import DIF

import sys
import os
sys.path.append(os.path.abspath("../../deep-i-forest/deep-iforest"))
from algorithms.dif import DIF

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

In [28]:
data = pd.read_csv('vacuum_sensor_data.csv', sep = ';')
data

,momento,1-0005,5-0005,1-_006,5-_006,1-_007,5-_007,1-_008,5-_008,1-_010,...,1-_098,5-_098,1-_099,5-_099,1-0112,5-0112,1-0116,5-0116,1-0109,5-0109
0,1,-588.88,-580.21,-590.50,-582.25,-593.51,-585.46,-595.51,-587.47,-575.36,...,-588.17,-581.42,-594.26,-586.26,-589.46,-581.75,-592.92,-586.26,-593.93,-586.30
1,2,-588.38,-579.71,-590.80,-582.46,-593.80,-585.67,-595.55,-587.59,-574.11,...,-588.34,-581.63,-594.51,-586.26,-589.29,-581.58,-592.80,-586.26,-594.13,-586.51
2,3,-584.41,-575.74,-585.83,-577.66,-591.38,-583.21,-589.84,-578.04,-563.34,...,-588.17,-581.33,-592.88,-584.34,-588.50,-580.62,-590.21,-583.75,-593.68,-585.84
3,4,-573.73,-565.27,-570.31,-562.81,-578.91,-571.24,-578.61,-565.90,-555.00,...,-584.79,-576.87,-580.62,-568.15,-586.96,-578.75,-578.66,-571.19,-590.88,-582.63
4,5,-567.27,-558.55,-562.26,-554.55,-570.69,-562.93,-567.18,-557.51,-545.99,...,-579.11,-570.99,-572.11,-560.43,-583.45,-574.91,-571.06,-563.56,-587.38,-578.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,637,-237.18,-253.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,638,-237.10,-253.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,639,-237.14,-253.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,640,-237.10,-253.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#rename column to have consistent naming 
import re

def clean_column(col):
    if col == 'momento':
        return col
    match = re.match(r"(\d)-_?(\d+)", col)
    if match:
        sensor, comp = match.groups()
        return f"{sensor}-{int(comp):04d}"
    return col  # fallback in case format is already correct

# Apply renaming
data.columns = [clean_column(col) for col in data.columns]

In [30]:
# drop rows(seconds) where data from any sensor is missing
data=data.dropna(axis=0)

In [5]:
data.shape

(533, 77)

In [31]:
#split columns into train and validation
defective_columns = ['1-0116', '5-0116', '1-0109', '5-0109']
normal_control_columns = ['1-0008', '5-0008', '1-0064', '5-0064']
validation_columns = defective_columns + normal_control_columns

df_validation = data[['momento'] + validation_columns]
train_columns = [col for col in data.columns if col not in validation_columns]
df_train = data[train_columns]



In [32]:

from sklearn.metrics import confusion_matrix, classification_report
from itertools import product
import torch



# --- 2. Prepare training data ---

# Drop 'momento', transpose so each row = one component signal over time
X_train_raw = df_train.drop(columns=['momento']).T  # shape: (num_components, n_timesteps)

# Scale each component (row) independently
def scale_rows(X):
    return np.array([StandardScaler().fit_transform(row.reshape(-1, 1)).flatten() for row in X])

X_train_scaled = scale_rows(X_train_raw.values)
n_timesteps_train = X_train_scaled.shape[1]

# --- 3. Train DIF model ---
dif = DIF(
    n_ensemble=100,
    n_hidden=[500, 100],
    n_emb=20,
    activation='tanh',
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=1
)
dif.fit(X_train_scaled)

# --- 4. Prepare labels for validation components ---
label_map = {col: 1 if col in defective_columns else 0 for col in validation_columns}
y_val = np.array([label_map[col] for col in validation_columns])

# --- 5. Grid search over time windows and thresholds ---
# We use only windows matching train timesteps count exactly.

momento_values = np.sort(data['momento'].unique())
window_size = n_timesteps_train

# Generate candidate windows of length window_size in the momento timeline
window_starts = momento_values[:-window_size + 1]
window_ends = window_starts + window_size - 1

window_ranges = list(zip(window_starts, window_ends))

threshold_percentiles = [90, 92, 94, 95, 96, 98]

results = []

for (start, end), perc in product(window_ranges, threshold_percentiles):
    # Select validation window
    mask = (df_validation['momento'] >= start) & (df_validation['momento'] <= end)
    df_val_window = df_validation.loc[mask, ['momento'] + validation_columns]

    if df_val_window.shape[0] != window_size:
        # Skip windows that don't match exact length (sanity check)
        print(f"Skipping window {start}-{end} due to length mismatch: val {df_val_window.shape[0]} vs train {window_size}")
        continue

    # Prepare validation features
    X_val_raw = df_val_window.drop(columns=['momento']).T.values  # shape: (num_components, window_size)

    # Scale validation data **using training scalers independently for each component**
    # Note: scaling each row of validation with its own scaler fitted on that row is wrong here,
    # so instead we standard scale using the same means/stds as training per component.

    # For each component, fit scaler on training row, then transform validation row:
    X_val_scaled = np.empty_like(X_val_raw)
    for i in range(X_val_raw.shape[0]):
        scaler = StandardScaler()
        scaler.fit(X_train_raw.values[i].reshape(-1, 1))
        X_val_scaled[i] = scaler.transform(X_val_raw[i].reshape(-1, 1)).flatten()

    # Compute anomaly scores
    scores = dif.decision_function(X_val_scaled)

    # Threshold and predict anomalies
    threshold = np.percentile(scores, perc)
    preds = (scores >= threshold).astype(int)

    # Evaluate
    cm = confusion_matrix(y_val, preds)
    cr = classification_report(y_val, preds, zero_division=0, output_dict=True)

    results.append({
        'window': (start, end),
        'threshold_percentile': perc,
        'confusion_matrix': cm,
        'classification_report': cr,
        'accuracy': cr['accuracy'],
        'precision_defective': cr['1']['precision'],
        'recall_defective': cr['1']['recall'],
        'f1_defective': cr['1']['f1-score'],
    })

# --- 6. Select best result by F1 score for defective class ---
if len(results) == 0:
    print("No valid results found (all windows skipped or no predictions).")
else:
    best_result = max(results, key=lambda r: r['f1_defective'])
    print("Best result:")
    print(f"Window: {best_result['window']}")
    print(f"Threshold percentile: {best_result['threshold_percentile']}")
    print("Confusion Matrix:\n", best_result['confusion_matrix'])
    print("Classification Report:")
    print(pd.DataFrame(best_result['classification_report']).transpose())


network additional parameters: {'n_hidden': [500, 100], 'n_emb': 20, 'skip_connection': None, 'dropout': None, 'activation': 'tanh', 'be_size': 100}
training done, time: 1.3
Best result:
Window: (1, 533)
Threshold percentile: 90
Confusion Matrix:
 [[3 1]
 [4 0]]
Classification Report:
              precision  recall  f1-score  support
0              0.428571   0.750  0.545455    4.000
1              0.000000   0.000  0.000000    4.000
accuracy       0.375000   0.375  0.375000    0.375
macro avg      0.214286   0.375  0.272727    8.000
weighted avg   0.214286   0.375  0.272727    8.000
